# Project : Building data model and writing ETL job (python+Mysql)  

In [ ]:
!pip install mysql-connector-python
!pip install pandas 

### import libraries

In [2]:
import mysql.connector as mc
import pandas as pd

### Create a connection to the default database 

In [3]:
# extra parameter added = auth_plugin='mysql_native_password(Error-Authentication plugin'caching_sha2_password'cannot be loaded:)

try:
    conn=mc.connect(host='localhost',user='root',passwd='Admin@123',database='Mysql',auth_plugin='mysql_native_password')
    print('Connection done successfully')
except Exception as e:
    print(e)

Connection done successfully


### get the cursor using connection 

In [4]:
try:
    cur=conn.cursor()
except Exception as e:
    print(e)

### set autocommit true 

In [5]:
conn.autocommit=True

### create DB

In [7]:
try:
    cur.execute('create database datamodel')
    print('Database created successfully')
except Exception as e:
    print(e)

Database created successfully


In [43]:
cur.execute('show databases')
result=cur.fetchall()
for row in result:
    print(row,'\n')

('abc',) 

('bank',) 

('classicmodels',) 

('clause',) 

('coalesce',) 

('cte',) 

('datamodel',) 

('fun',) 

('information_schema',) 

('joins',) 

('myfirstdb',) 

('mysql',) 

('performance_schema',) 

('pivot',) 

('practice1',) 

('subquery',) 

('xyz',) 



### close the current default connection and reconnect on newly created DB 

In [8]:
conn.close()

In [10]:
# let's establish reconnection on new DB and get the cursor

try:
    conn=mc.connect(host='localhost',user='root',passwd='Admin@123',database='datamodel',auth_plugin='mysql_native_password')
    print('Connection done successfully on new database')
except Exception as e:
    print(e)

try:
    cur=conn.cursor(buffered=True)
except Exception as e:
    print(e)

conn.autocommit=True

Connection done successfully on new database


### import CSV files 

In [12]:
# PassengerId = id of the passenger
# Survived = 1-YES 0-NO

gender_submission_dataset=pd.read_csv('C:\\Users\\Admin\\OneDrive\\Desktop\\DM_project\\gender_submission.csv')
gender_submission_dataset.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [13]:
train_dataset=pd.read_csv('C:\\Users\\Admin\\OneDrive\\Desktop\\DM_project\\train.csv')
train_dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [14]:
# Pclass = 1-first_class 2-second_class 3-third_class
# Embarked = Boarding station [S-Southampton C-Cherbourg Q-Queenstown]

clean_TrainDataset=train_dataset[['PassengerId','Pclass','Name','Sex','Age','Ticket','Embarked']]
clean_TrainDataset.head()

,PassengerId,Pclass,Name,Sex,Age,Ticket,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,S
1,2,1,"Cumings, Mrs. John Bradley",female,38.0,PC 17599,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,STON/O2. 3101282,S
3,4,1,"Futrelle, Mrs. Jacques Heath",female,35.0,113803,S
4,5,3,"Allen, Mr. William Henry",male,35.0,373450,S


In [15]:
test_dataset=pd.read_csv('C:\\Users\\Admin\\OneDrive\\Desktop\\DM_project\\test.csv')
test_dataset.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [16]:
# Sib Sp = Number of Siblings/Spouses Aboard
# Parch = Number of Parents/Children Aboard

clean_TestDataset=test_dataset.drop(['Pclass','Name','Sex','Age','Ticket','Fare','Cabin'], axis=1)
clean_TestDataset.head()

,PassengerId,SibSp,Parch,Embarked
0,892,0,0,Q
1,893,1,0,S
2,894,0,0,Q
3,895,0,0,S
4,896,1,1,S


### create data model based on above data and

### create tables  

In [17]:
try:
    cur.execute('create table gender_submission(PassengerId int primary key,Survived int)')
    print('Table gender_submission created successfully')
except Exception as e:
    print(e)

Table gender_submission created successfully


In [19]:
try:
    cur.execute('create table train(PassengerId int(20),Pclass int(10),Name varchar(100),Sex varchar(50),Age float,Ticket varchar(100),Embarked char(50))')
    print('Table train created successfully')
except Exception as e:
    print(e)

Table train created successfully


In [24]:
try:
    cur.execute('create table test(PassengerId int(20),SibSp int(20),Parch int(20),Embarked char(20))')
    print('Table test created successfully')
except Exception as e:
    print(e)

Table test created successfully


### alter-chnage in table structure

In [7]:
cur.execute('alter table test ADD FOREIGN KEY (PassengerId) REFERENCES gender_submission(PassengerId);')
print('foreign key added')

foreign key added


### insert the csv files data into tables

In [25]:
try:
    cur.execute('insert into gender_submission(PassengerId,Survived) values(892,0),(893,1),(894,0),(895,0),(896,1)')
    print('data inserted successfully')
except Exception as e:
    print(e)

data inserted successfully


### validate data that was inserted or not into the table 

In [39]:
cur.execute('select * from gender_submission')
result=cur.fetchall()
for row in result:
    print(row,'\n')

(892, 0) 

(893, 1) 

(894, 0) 

(895, 0) 

(896, 1) 



In [33]:
try:
    cur.execute("insert into train(PassengerId,Pclass,Name,Sex,Age,Ticket,Embarked) values(1,3,'Braund,Mr.Owen Harris','male',22.0,'A/5 21171','S'),(2,1,'Cumings,Mrs.John Bradley','female',38.0,'PC 17599','C'),(3,3,'Heikkinen,Miss.Laina','female',26.0,'STON/O2 3101282','S'),(4,1,'Futrelle,Mrs.Jacques Health','female',35.0,'113803','S'),(5,3,'Allen,Mr.William Henry','male',35.0,'373450','S')")
    print('data inserted successfully')
except Exception as e:
    print(e)

data inserted successfully


In [37]:
cur.execute('select * from train')
result=cur.fetchall()
for row in result:
    print(row,'\n')

(1, 3, 'Braund,Mr.Owen Harris', 'male', 22.0, 'A/5 21171', 'S') 

(2, 1, 'Cumings,Mrs.John Bradley', 'female', 38.0, 'PC 17599', 'C') 

(3, 3, 'Heikkinen,Miss.Laina', 'female', 26.0, 'STON/O2 3101282', 'S') 

(4, 1, 'Futrelle,Mrs.Jacques Health', 'female', 35.0, '113803', 'S') 

(5, 3, 'Allen,Mr.William Henry', 'male', 35.0, '373450', 'S') 



In [32]:
try:
    cur.execute("insert into test(PassengerId,SibSp,Parch,Embarked) values(892,0,0,'Q'),(893,1,0,'S'),(894,0,0,'Q'),(895,0,0,'S'),(896,1,1,'S')")
    print('data inserted successfully')
except Exception as e:
    print(e)

data inserted successfully


In [44]:
cur.execute('select * from test')
result=cur.fetchall()
for row in result:
    print(row,'\n')

(892, 0, 0, 'Q') 

(893, 1, 0, 'S') 

(894, 0, 0, 'Q') 

(895, 0, 0, 'S') 

(896, 1, 1, 'S') 



### update - updating some data values

In [23]:
cur.execute("""update train
            set Ticket = (case when PassengerId = 4 then 'A/5 113803'
                               when PassengerId = 5 then 'PC 373450'
                         end)
            where PassengerId in(4,5);""")
print("Data update")

Data update


In [7]:
cur.execute('select * from train')
result=cur.fetchall()
for row in result:
    print(row,'\n')

(1, 3, 'Braund,Mr.Owen Harris', 'male', 22.0, 'A/5 21171', 'S') 

(2, 1, 'Cumings,Mrs.John Bradley', 'female', 38.0, 'PC 17599', 'C') 

(3, 3, 'Heikkinen,Miss.Laina', 'female', 26.0, 'STON/O2 3101282', 'S') 

(4, 1, 'Futrelle,Mrs.Jacques Health', 'female', 35.0, 'A/5 113803', 'S') 

(5, 3, 'Allen,Mr.William Henry', 'male', 35.0, 'PC 373450', 'S') 



### finally close cursor and connection

In [13]:
cur.close()
conn.close()
print('Database connectivity disbale')

Database connectivity disbale
